### 安裝 jieba 套件

In [ ]:
!pip install jieba

### 建立新目錄，用來存放等等下載的辭典檔案

In [ ]:
!mkdir jieba_data

### 下載對繁體分詞更好的辭典檔 (jieba原廠提供)
`dict.txt.big`

In [ ]:
!wget https://github.com/fxsjy/jieba/raw/master/extra_dict/dict.txt.big -O jieba_data/dict.txt.big

-----------------------------------------------

### 檢查 Jieba 版本

In [1]:
import jieba
jieba.__version__

'0.42.1'

### 指定辭典檔

In [2]:
import jieba
jieba.set_dictionary('jieba_data/dict.txt.big')

In [3]:
#看辭典 前10行內容 (詞項.詞頻.詞性)
!head -10 jieba_data/dict.txt.big

1号店 3 n
1號店 3 n
4S店 3 n
4s店 3 n
AA制 3 n
AB型 3 n
AT&T 3 nz
A型 3 n
A座 3 n
A股 3 n


# jieba分詞

In [4]:
#查詢參數
?jieba.cut  

### 精確模式分詞 `(cut_all=False)` 預設
試圖將句子最精確地切開，適合文本分析

In [5]:
# 目標： 斷成 今天/天氣/真好

text_str = '今天天氣真好'

seg_result = jieba.cut(text_str, cut_all=False) 

print(' / '.join(list(seg_result))) #斷詞結果要放進list才看的到

Building prefix dict from /home/jovyan/work/06 Text_wordcloud/jieba_data/dict.txt.big ...
Loading model from cache /tmp/jieba.ua7fe3492a0c9aeb6416c4783f2c60a50.cache
Loading model cost 2.208 seconds.
Prefix dict has been built successfully.


今天天氣 / 真 / 好


### 全模式分詞 `(cut_all=True)`
把句子中所有的可以成詞的詞語都掃描出來，速度非常快，但是不能解決歧義

In [6]:
seg_result = jieba.cut(text_str, cut_all=True)

print(' / '.join(list(seg_result)))

今天 / 今天天氣 / 天天 / 天氣 / 真好


### 搜尋引擎模式分詞
在精確模式的基礎上，對長詞再次切分，提高召回率，適合用於搜尋引擎分詞

In [7]:
#查詢參數
?jieba.cut_for_search 

In [8]:
seg_result = jieba.cut_for_search(text_str, HMM=True)

print(' / '.join(list(seg_result)))

今天 / 天天 / 天氣 / 今天天氣 / 真 / 好


In [9]:
seg_result = jieba.cut_for_search(text_str, HMM=False)
print(' / '.join(list(seg_result)))

今天 / 天天 / 天氣 / 今天天氣 / 真 / 好


### Paddle 模式分詞 (use_paddle=True)
`常常會讓kernel die`
- 利用PaddlePaddle分度學習框架，訓練序列標註網路模型分詞。同時支援雌性標註。
- Paddle模式使用時須安裝paddlepaddle-tiny( pip install paddlepaddle-tiny==1.6.1 )，使用Paddle模式時請注意
- Jieba必須是 v0.40及以上版本，升級 Jieba ( pip install jieba --upgrade )
- `目前似乎只支援python3.7版本`

In [ ]:
!pip install paddlepaddle-tiny==1.6.1

In [ ]:
jieba.enable_paddle() #啟動
seg_result = jieba.cut(text_str, use_paddle=True)
print(' / '.join(list(seg_result)))

#### Paddle 詞性標註
- 詞性說明: https://gist.github.com/luw2007/6016931

In [ ]:
import jieba.posseg as pseg
jieba.enable_paddle()
seg_result = pseg.lcut(text_str, use_paddle=True)
for w, p in seg_result:
    print("%s, %s"%(w, p))

-----------------------------------------

# 自定義詞庫 (把哪些字設為一個詞，不被切開)

In [10]:
# 已先自創文字檔 並將自訂義詞寫入
!cat jieba_data/mydict.txt

是在哈囉

In [13]:
#載入詞庫
jieba.load_userdict('jieba_data/mydict.txt')

text_str_2 = '是在哈囉嗎?'

# 用.lcut 可切開後直接產生list
seg_result = jieba.lcut(text_str_2)

print(' / '.join(seg_result))

是在哈囉 / 嗎 / ?


# 停止字 (過濾掉不想要的詞)
文章裡如果有這些詞，就不讓它出現

In [ ]:
#下載 示範辭典檔案 stop_words.txt
!wget https://raw.githubusercontent.com/fxsjy/jieba/master/extra_dict/stop_words.txt -O jieba_data/stop_words.txt

In [20]:
# 開檔後一次讀一行，並加入list

stop_words_list = []

with open(file='jieba_data/stop_words.txt',mode='r', encoding="UTF-8") as file:
    for line in file:
        line = line.strip() #把空白過濾掉
        stop_words_list.append(line)
        
print(stop_words_list)

['the', 'of', 'is', 'and', 'to', 'in', 'that', 'we', 'for', 'an', 'are', 'by', 'be', 'as', 'on', 'with', 'can', 'if', 'from', 'which', 'you', 'it', 'this', 'then', 'at', 'have', 'all', 'not', 'one', 'has', 'or', 'that', '的', '了', '和', '是', '就', '都', '而', '及', '與', '著', '或', '一個', '沒有', '我們', '你們', '妳們', '他們', '她們', '是否']


In [16]:
# 開檔後一次讀完，再進行切割

with open(file='jieba_data/stop_words.txt',mode='r', encoding="UTF-8") as file:
    
    #依照換行字元 \n 進行切割，切完為 list
    stop_words = file.read().split('\n')
    
    print(type(stop_words))
    print(stop_words)

<class 'list'>
['the', 'of', 'is', 'and', 'to', 'in', 'that', 'we', 'for', 'an', 'are', 'by', 'be', 'as', 'on', 'with', 'can', 'if', 'from', 'which', 'you', 'it', 'this', 'then', 'at', 'have', 'all', 'not', 'one', 'has', 'or', 'that', '的', '了', '和', '是', '就', '都', '而', '及', '與', '著', '或', '一個', '沒有', '我們', '你們', '妳們', '他們', '她們', '是否']


In [18]:
text_str_3 = '我是一位小學生，從小學習鋼琴，希望成為youtuber'

seg_result = jieba.lcut(text_str_3)
print(seg_result)

seg_result_stopword = []
for term in seg_result:
    if term not in stop_words:
        seg_result_stopword.append(term)
        
seg_result_stopword

['我', '是', '一位', '小學生', '，', '從小', '學習', '鋼琴', '，', '希望', '成為', 'youtuber']


['我', '一位', '小學生', '，', '從小', '學習', '鋼琴', '，', '希望', '成為', 'youtuber']

## 其他

In [19]:
print(jieba.DEFAULT_DICT_NAME)
# check jieba/jieba/__init__.py

dict.txt
